# Métodos predictivos: tarea de asignación (semana 5)
## Implementación de clasificadores basados en ensembles.

## Instrucciones
En este notebook encontrarás los pasos necesarios para realizar la tarea de la 5ª semana de Métodos Predictivos del Máster en Ciencia de Datos. Lea detenidamente y siga los pasos indicados en las siguientes celdas, complete el código donde se indique ``# COMPLETAR AQUI``, respetando el formato o los nombres de funciones especificados.

## Descripción de la tarea
En esta tarea, el principal objetivo será implementar métodos basados en ensemble, utilizando distintos métodos de combinación, así como analizar su comportamiento y resultados en un estudio experimental. La tarea consta de varios apartados:
0. Carga y preparación de datos
1. Combinación por voto mayoritario (0.5 punto)
2. Combinación de probabilidades (0.5 punto)
3. Creación de ensemble tipo *bagging* (2.5 puntos)
4. Estudio experimental y análisis (1.5 puntos)

**NOTA**: A lo largo de toda la tarea se proponen distintas funciones con ciertos parámetros. Cualquier parámetro de cualquiera de las funciones que considere oportuno añadir, modificar, o eliminar, puede hacerlo siempre que justifique su elección correctamente.

Rellenar esta celda con los datos del alumno

**Nombre**: 

**Apellidos**: 

## 0. Carga y preparación de datos

En primer lugar, en esta sección el estudiante debe cargar los datos que utilizará a lo largo del *notebook*. En este caso, no se indica o restringe a utilizar unos datos concretos, sino que el estudiante puede escoger los datos que considere, siempre y cuando se trate de conjuntos de datos para clasificación. La única restricción es no utilizar un conjunto de datos de los utilizados en *notebooks* anteriores de la asignatura.

Se pueden utilizar conjuntos de datos tanto de clasificación binaria como multi-clase. Sin embargo, por simplicidad de implementación, se recomienda utilizar de clasificación binaria (aunque si el estudiante desea utilizar multi-clase, no hay problema). En caso de escoger problemas binarios, puede: 1) utilizar directamente un conjunto de datos binario; o 2) cargar un conjunto de datos multi-clase y filtrar sus patrones para dejar unicamente aquellos pertenecientes a dos clases, convirtiéndolo en un problema binario más pequeño.

Una vez cargados los datos, el estudiante deberá considerar si necesita realizar algún preprocesado de los mismos (si lo considera necesario), y deberá realizar una partición de los datos en entrenamiento y test.

Además, sería oportuno imprimir por pantalla algunas características de los datos, como el número de patrones de entrenamiento/test y el número de clases distintas.

In [1]:
# Carga de datos, preprocesado (si es necesario), y partición en train-test

# COMPLETAR AQUI
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

data = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data", sep=",")
data
y = data["R"]

X = data.drop(["R"], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [2]:

print("El número de patrones para el conjunto de entremanieto es: {0}".format(len(X_train)))
print("El número de patrones para el conjunto de test es: {0}".format(len(X_test)))

El número de patrones para el conjunto de entremanieto es: 138
El número de patrones para el conjunto de test es: 69


## 1. Combinación por voto mayoritario (0.5 puntos)

En primer lugar, implementaremos una función para obtener la decisión final del ensemble para un patrón dado, por el método de voto mayoritario. En este caso, la salida debe ofrecer no la clase categórica, sino la probabilidad de pertenencia a la clase considerada positiva (ratio de predicciones positivas entre el total).

La función debe seguir el siguiente prototipo: ``voto_mayoritario(predicciones, pos_label)``; donde el parámetro ``predicciones`` será una lista con los valores **categóricos** de predicción de cada uno de los clasificadores base, y ``pos_label`` será el valor de clase considerado como clase positiva. La función debe devolver un único valor, que sea la probabilidad predicha de pertenencia a la clase considerada positiva

Por ejemplo, si la función recibe la lista ``['P', 'N', 'N', 'P', 'P']``, (y el parámetro ``pos_label='P'``) debe devolver ``0.6``, que sería la probabilidad de pertenencia a la clase positiva (ratio de predicciones positivas entre el total).




In [3]:
def voto_mayoritario(predicciones, pos_label="R"):
  '''
  Método de combinación para el ensemble para un patrón por voto mayoritario
    de las predicciones de los clasificadores base.
  Devuelve la predicciones de pertenencia a la clase positiva, calculada como el
    ratio de probabilidades de la clase positiva entre el total de predicciones

  :param predicciones: Lista con valores categóricos con las predicciones de los distintos clasificadores base
  :param pos_label: Valor considerado la clase positiva. Por defecto se considera el valor 1.
   
  :return: Probabilidad predicha de pertenencia a la clase positiva
  '''
  # COMPLETAR AQUI

  num_true_positive = 0
  
  for pre in predicciones:

    if pre == pos_label:

      num_true_positive += 1
  
  return num_true_positive/len(predicciones)

## 2. Combinación de probabilidades (0.5 puntos)
En esta sección implementaremos otra función de combinación de predicciones, para clasificadores que proporcionan probabilidades en lugar de únicamente la clase categórica.

La función debe seguir el siguiente prototipo: ``comb_probabilidades(proba)``; donde el parámetro ``proba`` será una lista con las probabilidades predichas de pertenencia a la clase positiva por cada uno de los clasificadores base. La función debe devolver un único valor, que sea el de la probabilidad estimada por el ensemble de pertenencia a la clase positiva.

Por ejemplo, si la función recibe la lista ``[0.3, 0.6, 0.8, 0.45, 0.7]``, debe devolver 0.57, que es la media de dichos valores.

In [4]:
def comb_probabilidades(proba):
  '''
  Devuelve la probabilidad de pertenencia a la clase positiva predicha por el 
    ensemble para un patrón, a partir de las probabilidades predichas por cada 
    clasificador base.

  :param proba: Lista con las probabilidades de pertenencia a la clase positiva predichas por cada clasificador base
   
  :return: Probabilidad de pertenencia a la clase positiva predicha por el ensemble
  '''
  # COMPLETAR AQUI
  import numpy as np
  
  return np.mean(proba)

## 3. Creación de ensemble tipo bagging (2.5 puntos)

En esta sección, vamos a crear nuestro método de generación de ensembles, basado en el enfoque *bagging*. Además, para crear nuestro clasificador de ensemble, vamos a seguir la estructura sugerida por la librería *scikit-learn* para construir nuevos clasificadores compatibles con la propia librería. Aunque a continuación en la celda de implementación se dejan anotaciones para poder implementarlo correctamente, es recomendable tener en cuenta la [Guía de scikit-learn para el desarrollo de clasificadores propios](https://scikit-learn.org/stable/developers/develop.html).

La selección del tipo clasificador base (knn, svm, árbol de decisión, ...) a utilizar dentro del ensemble, así como sus parámetros, serán decisión del alumno. Cualquier opción es válida, siempre que sea un clasificador de *scikit-learn* capaz de producir probabilidades como salida.

En la siguiente celda, complete el código necesario para implementar el ensemble como clasificador de scikit-learn. Por defecto, se crea un clasificador llamado ``TemplateClassifier``. El estudiante debe modificar el nombre de la clase para darle a su clasificador el nombre que considere oportuno. 

A su vez, el clasificador debe contener, como mínimo, 4 funciones en su interior:
*   ``__init__``: Recibe como parámetros todos los parámetros necesarios para crear el modelo. En nuestro caso debe recibir, como mínimo: el número de clasificadores base del ensemble, el tipo de combinación a utilizar (voto mayoritario o combinacion de probabilidades), el ratio de instancias a utilizar para entrenar cada clasificador base, la etiqueta de clase considerada como la clase positiva, y una variable ``random_state`` que actúe como semilla para números aleatorios para el entrenamiento posterior del modelo. 
  *  **IMPORTANTE:** Debe completar, o en esta misma celda de texto, o como documentación de la función en el código, qué significa cada uno de los parámetros que recibe la función, y que valores podría tomar.
*   ``fit``: Esta función debe realizar los pasos necesarios para entrenar el modelo. Además de los pasos ya escritos dentro de la función, debe hacer lo necesario para entrenar el ensemble (es decir, entrenar cada uno de los clasificadores del ensemble).
*   ``predict``: Esta función será la encargada de proporcionar predicción de clase categórica para cada patrón del conjunto de datos recibido. Consideramos por defecto un umbral de 0.5; es decir, si la función de combinación (voto mayoritario o combinación de predicciones) devuelve una probabilidad mayor o igual a 0.5, se predice la clase positiva, y la negativa en caso contrario. La salida debe ser una lista de valores categóricos que correspondan con la predicción de clase.
*   ``predict_proba``: Esta función será la encargada de proporcionar la predicción de pertenencia a la clase positiva en forma de probabilidad, para cada patrón del conjunto de datos recibido.


In [18]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import euclidean_distances
import numpy as np
from sklearn import tree
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

class BaggingEnsembleModel():
  def __init__(self, n_models=10, combination='voting', ratio=0.7, pos_label="R", negative_label="M", random_state=0):
    
    # Asignar el parámetro n_models como parte de la clase
    self.n_models = n_models
    
    # La funcion debe recibir más parámetros; este es solo un ejemplo.
    #   Modifique la lista de parámetros de la función a lo especificado 
    #   anteriormente.

    # COMPLETAR AQUI FUNCIÓN __init__
    self.combination = combination
    
    self.ratio = ratio

    self.pos_label = pos_label

    self.y_ = negative_label

    self.random_state = random_state

    self.models = []

    # La función __init__ no devuelve nada
    

  def fit(self, X, y):
    # Comprobar que X e y tienen la estructura correcta
    X, y = check_X_y(X, y)

    # Almacenar los valores de clase durante el entrenamiento
    self.classes_ = unique_labels(y)

    # COMPLETAR AQUI FUNCIÓN fit --> Entrenamiento de clasificadores base
    #   Para ello tenga en cuenta que debería considerar, al menos, los 
    #   parámetros: n_models, ratio, y random_state

    for bag in range(self.n_models):

      muestra = np.random.choice(np.arange(X.shape[0]), size = X.shape[0], replace=True)

      X_bag = X[muestra]

      y_bag = y[muestra]

      model = SVC(C=self.ratio, random_state=self.random_state, probability=True)

      model.fit(X_bag, y_bag)
      
      self.models.append(model)

    # La función fit debe devolver siempre el propio clasificador (self)
    return self


  def predict(self, X):
    # Comprobar si se ha llamado a la función fit antes de a predict
    check_is_fitted(self)

    # Comprobar la estructura de entrada
    X = check_array(X)

    # Obtener la clase negativa (en escenario binario)
    neg_label = list(set(np.unique(self.y_)) - set([self.pos_label]))[0]

    # COMPLETAR AQUI FUNCIÓN predict
    #   Para cada patrón de test debe combinar las salidas de los clasificadores
    #   base, y proporcionar la salida final

    ensemble = pd.DataFrame()
    final_predict = []

    # Predict over all models
    for i, model in enumerate(self.models):

      if self.combination == "voting":

        ensemble["M" + str(i)] = model.predict(X)

      else:

        ensemble.append(model.predict_proba(X))
      
    for index, row in ensemble.iterrows():
      
      if self.combination == "voting":

        pred = voto_mayoritario(row)
      
      else:

        pred = comb_probabilidades(row)

      
      if pred >= 0.5:
        final_predict.append(self.pos_label)

      else:
        final_predict.append(neg_label)

    # La función predict debe devolver una lista de valores categóricos
    return final_predict


  def predict_proba(self, X):
    # COMPLETAR AQUI FUNCIÓN predict_proba

    predict_proba = self.model.predict_proba(X)
    # La función predict_proba debe devolver una lista de probabilidades

    return predict_proba


## 4. Estudio experimental y análisis (1.5 puntos)

En esta sección, se pretende que el estudiante **implemente** un estudio experimental, **analice** sus resultados y **comente** los resultados y conclusiones obtenidas. Para ello, puede utilizar una o varias celdas de código, como considere que queda más limpio. También puede utilizar celdas de texto si lo considera necesario para el análisis de resultados.

Se pretende que el estudio experimental sirva para **analizar el comportamiento del ensemble** en cuanto a **varios aspectos**: número de clasificadores base, método de combinación, y ratio de instancias utilizadas para entrenar cada modelo.

Tras analizar esos resultados, debe escoger la mejor combinación de esos parámetros, y compararlo con un modelo que sea del mismo tipo y parámetros que los clasificadores base del ensemble (por ejemplo, si el ensemble utiliza árboles de decisión de clasificador base, comparar contra un único árbol de decisión), pero entrenado utilizando todo el conjunto de entrenamiento. Analice el rendimiento comparado de ambos métodos (ensemble y clasificador simple).

In [19]:
# COMPLETAR AQUI

model = BaggingEnsembleModel(n_models=10, combination='voting', ratio=0.7, pos_label="R", random_state=0)

model.fit(X_train, y_train)

model.predict(X_test)


,M0,M1,M2,M3,M4,M5,M6,M7,M8,M9
0,R,R,R,R,R,R,R,R,R,R
1,R,M,M,M,R,M,M,M,M,M
2,R,R,M,R,R,R,R,R,M,R
3,R,M,M,R,M,M,R,R,M,M
4,R,M,R,M,M,M,M,R,M,M
...,...,...,...,...,...,...,...,...,...,...
64,R,R,R,M,R,R,R,R,R,R
65,R,R,R,R,R,R,R,R,R,R
66,R,R,R,R,R,R,R,R,M,R
67,R,R,R,R,R,R,R,R,R,R
